In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from datasets import load_dataset
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from tqdm.auto import tqdm

import ast
import os
import pandas as pd

ModuleNotFoundError: No module named 'openai'

In [ ]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Set index name
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")

# Check if index exists and delete it
if pc.has_index(INDEX_NAME):
    pc.delete_index(INDEX_NAME)

# List of indexes
print("List of indexes:")
print(pc.list_indexes())


if not pc.has_index(INDEX_NAME):
    # Create index
    pc.create_index(
        name=INDEX_NAME, 
        dimension=model.get_sentence_embedding_dimension(), 
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        )
    )
    
    # Access index
    index = pc.Index(INDEX_NAME)
    print(index)

In [1]:

max_articles_num = 500
df = pd.read_csv('wiki.csv', nrows=max_articles_num)
df.head()

NameError: name 'pd' is not defined